# GPT 

На прошлой паре мы разбирали транформер в целом и попробовали дообучать BERT для задачи классификации. На этом посмотрим на другую популярную транформерную модель -  GPT (Generative pretrained transformer).

## Чем BERT и GPT похожи?
- Обе модели основаны на транформерных блоках с механизмом внимания и позиционным кодированием. Есть небольшие отличия в некоторых частях моделей (в gpt, например используется sparse attention), но они не принципиальные.

- В обоих моделях используется BPE токенизация. 
- Обе модели решают задачу языкового моделирования.

## Чем отличаются BERT и GPT?
- BERT - это энкодер-декодер модель (на прошлом семинаре на картинках изображалась именно такая модель). А GPT - это только декодер. Соответственно в GPT используется только self-attention, а в BERT и self-attention и cross-attention.
- BERT обучается на задачах Мasked language modelling (MLM) и Next Sentence Prediction (NSP). Это искусственные self supervised задачи: в MLM модель должна заполнить пропуски в тексте (например, "Я [MASK] в [MASK] один." -> "Я пошел в кино один"), а NSP - определить являются ли два предложения соседними. А GPT - это авторегрессионная модель, которая учится продолжать данную ей последовательностью ("Я пошел.." -> "Я пошел в" -> "Я пошел в кино" -> "Я пошел в кино один")
- BERT обычно предполгается дообучать под целевую задачу (например, классификацию), поэтому чаще всего от предобученной модели используется только encoder, поверх которого ставится новый слой под новую задачу. В GPT (особенно в последней версии огромных размеров) целевой задачей является именно генерация текста и поэтому предобученную модель либо вообще никак не дообучают, либо дообучают на генерацию нужных текстов (а не конкретную задачу вроде классификации). 

### Главная особенность GPT
Предобученную GPT можно использовать для решения целевой задачи (той же классификации) сразу из коробки, без дообучения, через генерацию текста. Нужно только сформулировать входную последовательность подходящим образом. Например, для определения класса текста можно подать в модель нужный текст и добавить в конце "Тема этого текста: ", на что модель выдаст продолжение "Новости" или "Спорт". Такой подход называется zero-shot learning. 
Еще есть few-shot learning, когда в модель подается что-то вроде "2+2=4, 1+3=4, 1+2= ", то есть модели показывается несколько примеров того, что от нее хотят и просят анологично продолжить последний пример. В этом случае также не происходит дообучения модели (веса не обновляются), поэтому на каждом запуске в модель нужно подавать нужный контекст. Такие контексты называются затравками (prompts). Подбирать их может быть достаточно сложно, даже есть такой термин "prompt engineering". (Есть способ обойти ручную подброку затравок, который называется prompt tuning. Про него можно почитать вот тут - https://habr.com/ru/company/yandex/blog/588214/) 

Также важная характеристика GPT моделей - их огромный размер. Самая большая модель GPT-3 имеет 175 млрд параметров, и она даже не выложена в открытый доступ, а предоставляется через API (https://openai.com/api/). Есть варианты поменьше, в том числе и для русского (RuGPT-3 от Сбера). Их можно попробовать в коллабе через huggingface. Давайте попробуем!



In [4]:
# !pip install transformers

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Загружаем модель ruGPT от сбера
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

In [18]:
# prompt engineering 
text = "Вопрос: 'Сколько будет 2+2?'\n Ответ: " # работает
text = "Вопрос: 'Сколько будет 3+3?'\n Ответ: " # не очень работает
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Вопрос: 'Сколько будет 3+3?'
 Ответ: 
3+3=5


## Методы генерации текста
В вводном семинаре про языковое моделирование мы уже пробовали генерировать текст и там мы сталкивались с вопросом - а как выбирать конкретный токен по вероятностям, которые выдает модель? Если просто использовать .argmax(), то генерация будет слишком детерменированной и будет много повторов. Обычно это не то, что нам нужно. GPT может применяться, например, для генерации ответов в чатботе и однобразность ответов - сильный минус.

На семинаре про базовое языковое моделирование мы решали это случайным семплированием из полученного распределения, но на практике используют немного другие подходы. Давайте на них посмотрим.

In [26]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется  '
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

### Argmax
Мы уже выше попробовали аргмакс, но давайте проверим его и на новом тексте, чтобы потом было с чем сравнивать

In [27]:
# если повторить запуск результат не изменится
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться  под  одеяло,  укрыться  от  дождя,  укрыться  от  него,  укрыться  от  него,  укрыться  от  него,  укрыться  от


### Beam Search

Чтобы разнообразить генерацию можно на каждом шаге выбирать не 1 наиболее вероятный, а N наиболее вероятных и генерировать продолжения для них всех.Такой подход называется beam search (поиск пучком). Параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. будет слишком много комбинаций на длинных текстах. Выбрать итоговый текст можно по общей вероятности, а можно просто рандомно.

In [41]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=False, num_beams=10, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться  под  одеяло  и  закрыть  глаза,  чтобы  не  видеть,  не  слышать,  не  чувствовать,  чтобы  не  видеть,  не  слышать,  не  чувствовать,  чтобы


Но это тоже не очень сильно помогает

### Сэмплирование с Температурой
Самый распространенный подход - семплирование с температурой. Это очень похоже на то, что мы делали на семинаре по языковому моделированию (np.random.choice(words, p=probas), но здесь добавляется параметр, который регулирует насколько случайно или детерменировано мы будем выбирать следующий токен. Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [44]:
out = model.generate(input_ids, do_sample=True, temperature=0.01, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться  под  одеяло,  укрыться  от  дождя,  укрыться  от  него,  укрыться  от  него,  укрыться  от  него,  укрыться  от


In [45]:
out = model.generate(input_ids, do_sample=True, temperature=0.3, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться   под   одеяло,   укрыться   от
дождя, укрыться от него.

- А что, если я не хочу прятаться?

- Тогда я не буду


In [46]:
out = model.generate(input_ids, do_sample=True, temperature=0.8, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   надеть  теплую  курточку.  С  утра  мы  были  в
"Сапога" поцарапана и вся в дырах. С утра и до обеда я


In [47]:
out = model.generate(input_ids, do_sample=True, temperature=1.5, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спать — но  из кухни сквозь тонкие стеклянные  жалюзи  тянет   чем‑то  несъедобным.  — Вредная еда
может испортить фигуру, я тебя


Видно, что чем больше температура, тем случайнее текст


Чтобы не выбирать совсем маловероятные слова, можно указать дополнительный параметр - top_k. Так семплирование будет происходить только из K наиболее вероятных токенов

In [51]:
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=1.2,
                     top_k=20,
                     max_length=50,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   в теплый  плед,  завернуться  в  одеяло,  как   маленький  мохнаток… Я  открыла  форточку…

…Я была в  полном   шоке


Температура большая, но текст все равно получается более менее тематически связанный

# Дообучение GPT

Чтобы генерировать тексты в нужном стиле/домене, можно дообучить GPT на своем корпусе. Давайте попробуем это сделать

In [92]:
# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc3440d25937fdac74506b73b76ee6e67f778a804756214363fc2a1a66ef
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/479aa59074c4dcd4c36106252da033d03bc92e3010947ce1d3714de224c2af1f.7362c0dbb32f750eeea5a5b93bbd0c6876eac41453369265d5a49df1c9142b6f
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/sberbank


Для наглядности результата возьмем текст песни Оксимирона

In [93]:
text = """Позитивная мотивация — явно не мой конёк
И мы все умрём, всё уже было до нас
Можно выдохнуть страх и уставить глаза в небосклон
Если окружающему блядству и сопротивляться, то не с печальным лицом
Всё повторится не раз, но мы живы сейчас, нами рано удобрять чернозём

Рэп спас мою жизнь, и не только J Dilla
Маленький еврей в чёрном жанре — Боб Дилан
Резкость Mobb Deep’а, лёгкость Q-Tip’а
Миксовал, мне сполна заплатили как Eric’у с Rakim’ом
Я верил, путь к Эврике найду
Хотел судьбу, красивую, как в крипе Rude с Эрика Баду
Сторителлинг на ходу
Чаксы, как у гейма, Тимбы, как у Наса, если ветер северный подул
Когда Талиб Квали ещё не выжил из ума
За много лет до конничива и ичима
Я эмси бичевал и линчевал
Всех, кто на лаврах почивал и их бездарными текстами ланчевал
10 лет явлением был Окс
Потом молчание, мучение, как у Вордул Мега из Cannibal Ox
Щас треки — мой порошок
Рэп спас мою жизнь, пришло время вернуть должок

Позитивная мотивация — явно не мой конёк
И мы все умрём
Всё уже было до нас, можно выдохнуть страх
И уставить глаза в небосклон
Если окружающему блядству и сопротивляться
То не с печальным лицом
Всё повторится не раз, но мы живы сейчас
Нами рано удобрять чернозём

Игра пошла не туда куда-то
Так вам говорят полтора фаната
Stop it, старая гвардия хип-хапа
“I love it when you call me Big Poppa”
Сами виноваты — жить самообманом
Зря вы забивали на скилл самопиара
Ваша проблема не в том, что рэп стал попсой
Но что ваша мифологема неконкурентоспособна
Ведь когда все под автотюном поют
Под ебаный тайп бит с ютуба немыслимую хуйню
Лучший момент, чтобы двигаться трендам наперекор
Бля, уверенно, хладнокровно, спроси у Кендрика с Коулом
Пол-десятилетия без Окси, я вылетал
Из чата, залогинился опять — а воз и ныне там
Рабы нытья, дитя, не пасуй, живёшь только дважды
Ведь если я способен вернуться, способен каждый

"""

В huggingface уже реализованы все инструменты для дообучения. Даже если вы не знакомы с pytorch разобраться будет не сложно

In [94]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file cached_lm_GPT2Tokenizer_64_train_dataset.txt [took 0.000 s]


## Обучение
Через класс Trainer реализовано все обучение, останется только запустить фит (точнее трейн в случае hg)

In [101]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",
    overwrite_output_dir=True, 
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [102]:
trainer.train()

***** Running training *****
  Num examples = 8
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.08056178092956542, metrics={'train_runtime': 34.3522, 'train_samples_per_second': 23.288, 'train_steps_per_second': 2.911, 'total_flos': 26129203200000.0, 'train_loss': 0.08056178092956542, 'epoch': 100.0})

In [109]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.1,
                        top_k=30,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
Нарушители спокойствия
Мои извинения, все в порядке
Все идет не так, как я ожидала
Все, все уже было до нас
Все повторится, но мы живы сейчас
Все плохо, все повторится снова
Но мы живы сейчас»

Рэп с tm’а’a
Чаксы, как у гейма
Наглый лесть и шлепок
Все, больше ничего не приходит на ум
Сами виноваты – жить как с психами
Зря вы так делаете
Все уже было до нас
Все уже было до нас
Все, все уже было когда-то
Все уже было и будет
Можно выдохнуть страх, если с ним не в ладу
Боль ушла, но боль останется, хотя я не верю
Все повторится, но мы живы сейчас»

Чаксы, как у гейма
Наглый лесть и шлепок
Все, больше ничего не приходит на


Результат не очень хороший, но можно попробовать добавить еще текстов и попробовать другие параметры семплирования